In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import csv
from batchmake import Batcher

In [2]:
datas= 'data_samples/'
word_to_id='data_samples/mapping.map'
with open(word_to_id, 'rb') as f:
    word_to_id = pickle.load(f)
inv_map = {v:k for k, v in word_to_id.items() }
# batch_data = datas+'output.txt.part0'
batch_data = datas+'preprocess.part0'
with open(batch_data, 'rb') as f:
    picked_data = pickle.load(f)

In [3]:
def attention(inputs, attention_size):
    inputs_shape = inputs.shape
    sequence_length = inputs_shape[1].value
    hidden_size = inputs_shape[2].value
    
    w_omega = tf.Variable(tf.random_normal([hidden_size,attention_size], stddev=0.1))
    b_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
    u_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
    
    # batch *k , attention_size
    v = tf.tanh(tf.matmul(tf.reshape(inputs, [-1,hidden_size]), w_omega)+
               tf.reshape(b_omega,[1,-1]))
    
    vu = tf.matmul(v, tf.reshape(u_omega,[-1,1])) # batch*k,1
    exps = tf.reshape(tf.exp(vu), [-1, sequence_length])  # (batch, k)
    alphas = exps / tf.reshape(tf.reduce_sum(exps,1), [-1,1])
    #(batch,k)
    output = tf.reduce_sum(inputs * tf.reshape(alphas, [-1, sequence_length,1]),1)
    # batch * sequence * hidden -> batch * hidden
    return output

In [4]:
hidden_size=32
sequence=5
input_dim=1
attention_size = 50
batch_size=16
grucell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
X = tf.placeholder(tf.float32, [None, sequence,input_dim])
Y = tf.placeholder(tf.float32, [None, sequence, input_dim])
with tf.variable_scope('RNN'):
    outputs, states = tf.nn.dynamic_rnn(grucell,X, dtype=tf.float32)

In [5]:
attention_output = attention(outputs, attention_size=attention_size)

In [6]:
filename_queue = tf.train.string_input_producer(['./inputs.csv'])
reader = tf.TextLineReader()
key, record_string = reader.read(filename_queue)
seq_lengths = [5]
print(key, record_string)
record_string = tf.reshape(record_string, [1,-1])
target_string = tf.slice(record_string, [1,0],[1,-1])
# file_content = tf.read_file(filename_queue)
# batch_x = tf.train.batch([record_string], batch_size=1)

Ses = tf.Session()

sequences, output = tf.contrib.training.bucket_by_sequence_length(input_length=seq_lengths,
                                             tensors=[record_string,target_string],
                                             batch_size=1,
                                             bucket_boundaries=[3],
                                             dynamic_pad=True
                                             )

Tensor("ReaderReadV2:0", shape=(), dtype=string) Tensor("ReaderReadV2:1", shape=(), dtype=string)


In [7]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=Ses, coord=coord)
Ses.run(tf.global_variables_initializer())
print(Ses.run([sequences,output]))

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Expected size[0] in [0, 0], but got 1
	 [[Node: Slice = Slice[Index=DT_INT32, T=DT_STRING, _device="/job:localhost/replica:0/task:0/cpu:0"](Reshape_6, Slice/begin, Slice/size)]]


OutOfRangeError: FIFOQueue '_2_bucket_by_sequence_length/bucket/top_queue' is closed and has insufficient elements (requested 1, current size 0)
	 [[Node: bucket_by_sequence_length/bucket/dequeue_top = QueueDequeueV2[component_types=[DT_INT32, DT_INT32, DT_STRING, DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](bucket_by_sequence_length/bucket/top_queue)]]

Caused by op 'bucket_by_sequence_length/bucket/dequeue_top', defined at:
  File "D:\python3_5anaconda\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\python3_5anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\python3_5anaconda\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "D:\python3_5anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\python3_5anaconda\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    

In [ ]:
print(Ses.run([sequences,output]))

In [6]:
keep_prob = 0.5
drop = tf.nn.dropout(attention_output, keep_prob=keep_prob)

w = tf.Variable(tf.truncated_normal([hidden_size,1], stddev=0.1))
b = tf.Variable(tf.constant(0., shape=[1]))
y_hat = tf.nn.xw_plus_b(drop,w,b)
y_hat = tf.squeeze(y_hat)

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_hat, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

Tensor("Sum_1:0", shape=(?, 32), dtype=float32)


In [7]:
s = tf.Session()

In [8]:
batcher = Batcher([batch_data], batch_size=batch_size, seq_length=sequence) #batchsize, sequence

for batch in batcher:
    for i, feed_data  in enumerate(batcher.sequence_iterator(batch)):
        inputs,targets,masks,identifiers,actual_lengths = feed_data
        print(inputs.shape , targets.shape, masks.shape, identifiers.shape,actual_lengths.shape)
        print(actual_lengths)
        
        feed_data = {X:inputs, Y:targets}
        #inputs :  batch, sequence
        
        lo, opt = s.run([loss,optimizer], feed_dict=feed_data)
        
            
    break

Loaded data partition data_samples/preprocess.part0 with 481 examples
[<pyreader.Container object at 0x000001C9A750A518>, <pyreader.Container object at 0x000001C9A78A1898>, <pyreader.Container object at 0x000001C9A79122B0>, <pyreader.Container object at 0x000001C9A79D2BA8>, <pyreader.Container object at 0x000001C9A7953358>, <pyreader.Container object at 0x000001C9A79803C8>, <pyreader.Container object at 0x000001C9A79EADD8>, <pyreader.Container object at 0x000001C9A7A06978>, <pyreader.Container object at 0x000001C9A7A12278>, <pyreader.Container object at 0x000001C9A7A12390>, <pyreader.Container object at 0x000001C9A7A124A8>, <pyreader.Container object at 0x000001C9A7A125C0>, <pyreader.Container object at 0x000001C9A7A126D8>, <pyreader.Container object at 0x000001C9A7A127F0>, <pyreader.Container object at 0x000001C9A7A12908>, <pyreader.Container object at 0x000001C9A7A12A20>]
(5, 16) (5, 16) (5, 16, 1) (16, 5) (16,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


ValueError: Cannot feed value of shape (5, 16) for Tensor 'Placeholder:0', which has shape '(?, 5, 1)'